#数据读取

In [14]:
#加载及安装包
import numpy as np #NumPy 通常与 SciPy（Scientific Python）和 Matplotlib
#（绘图库）一起使用， 这种组合广泛用于替代 MatLab，是一个强大的科学计算环境，
#有助于我们通过 Python 学习数据科学或者机器学习
import pandas as pd #Pandas 可以对各种数据进行运算操作，比如归并、再成形、选择，还有数据清洗和数据加工特征
#没有下面部分代码在加载statsmodels.api时会报错
import  scipy.signal.signaltools
def _centered(arr, newsize):
    # Return the center newsize portion of the array.
    newsize = np.asarray(newsize)
    currsize = np.array(arr.shape)
    startind = (currsize - newsize) // 2
    endind = startind + newsize
    myslice = [slice(startind[k], endind[k]) for k in range(len(endind))]
    return arr[tuple(myslice)]
scipy.signal.signaltools._centered = _centered   # pip install scipy==1.2 --upgrade
import statsmodels.api as sm
import seaborn as sbn # Seaborn 可实现对统计数据的可视化展示，基于 Python 语言开发，使用 matplotlib 库
#sklearn拥有可以用于监督和无监督学习的方法，一般来说监督学习使用的更多。
#sklearn中的大部分函数可以归为估计器(Estimator)和转化器(Transformer)两类。
#估计器(Estimator)其实就是模型，它用于对数据的预测或回归。基本上估计器都会
#有以下几个方法：
# fit(x,y) :传入数据以及标签即可训练模型，训练的时间和参数设置，数据集大小以及数据本身的特点有关
# score(x,y)用于对模型的正确率进行评分(范围0-1)。但由于对在不同的问题下，评判模型优劣的的标准不限于简单的正确率，可能还包括召回率或者是查准率等其他的指标，特别是对于类别失衡的样本，准确率并不能很好的评估模型的优劣，因此在对模型进行评估时，不要轻易的被score的得分蒙蔽。
# predict(x)用于对数据的预测，它接受输入，并输出预测标签，输出的格式为numpy数组。我们通常使用这个方法返回测试的结果，再将这个结果用于评估模型。
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
#安装并加载xgboost包
from xgboost import XGBClassifier
#安装并加载lightgbm包，在LightGBM提出之前，最有名的GBDT工具就是gbdtoost了，
#它是基于预排序方法的决策树算法。lightgbm是对gbdtoost的优化
from lightgbm import LGBMClassifier
#安装并加载catboost包。catboost也基于决策树。但是嵌入了自动将类别型特征处
#理为数值型特征的创新算法。
#Catboost采用排序提升的方法对抗训练集中的噪声点，从而避免梯度估计的偏
from catboost import CatBoostClassifier
from warnings import filterwarnings
filterwarnings("ignore")
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
# 加载数据
database = pd.read_csv("./纯磨玻璃结节LPA_整理.csv")
# 打印列名以检查
print(database.columns.tolist())
# 定义您想要选择的特征
features_to_select = [
    "ITH score", 
    "Size", 
    "Mean CT value", 
    "Pleural indentation", 
    "Age", 
    "Location", 
    "Shape"
]
database = pd.read_csv("./纯磨玻璃结节LPA_整理.csv")#同一级文件夹 './file.csv'   ##子目录文件夹'./subfolder/file.csv'  ##上级文件夹'../otherfolder/file.csv'  
rand_seed =10  #  数据划分的随机种子
ratio=0.3
X = database[features_to_select]
#数据集拆分
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= ratio, random_state= rand_seed)


['Location', 'Margin', 'Shape', 'Lobulation', 'Spiculation', 'Vascular convergence sign', 'Vacuole sign', 'Pleural indentation', 'Sex', 'Age', 'Size', 'Mean CT value', 'ITH score', 'Outcome']


In [16]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

# 定义超参数网格
params_xgb = {
    'n_estimators': [50, 100, 200],  # 树的数量
    'learning_rate': [0.01, 0.1, 0.2],  # 学习率
    'max_depth': [3, 5, 7],  # 树的最大深度
    'subsample': [0.6, 0.8, 1.0]  # 训练每棵树时使用的样本比例
}

# 初始化XGBoost分类模型
model_xgb = xgb.XGBClassifier(**params_xgb)

# 使用 GridSearchCV 进行 5 折交叉验证和超参数搜索
grid_search_xgb = GridSearchCV(
    model_xgb,
    params_xgb,
    cv=5,
    scoring=['accuracy', 'roc_auc'],
    refit='accuracy',
    n_jobs=-1,
    verbose=1
)

# 训练模型
grid_search_xgb.fit(X_train, y_train)

# 使用最优参数训练模型
best_model = grid_search_xgb.best_estimator_


Fitting 5 folds for each of 81 candidates, totalling 405 fits


In [17]:
import joblib# 保存模型
joblib.dump(best_model , 'XGBoost.pkl')

['XGBoost.pkl']